In [2]:
import pandas as pd
import numpy as np

In [28]:
Data = pd.read_csv("TMDB_movie_dataset_v11.csv")
Data= pd.DataFrame(Data)
Data.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [16]:
Data.isnull().sum()

id                            0
title                        13
vote_average                  0
vote_count                    0
status                        0
release_date             234198
revenue                       0
runtime                       0
adult                         0
backdrop_path            921710
budget                        0
homepage                1113290
imdb_id                  615912
original_language             0
original_title               13
overview                 266365
popularity                    0
poster_path              412757
tagline                 1068901
genres                   519618
production_companies     695224
production_countries     573578
spoken_languages         551131
keywords                 919551
dtype: int64

In [18]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1243317 entries, 0 to 1243316
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1243317 non-null  int64  
 1   title                 1243304 non-null  object 
 2   vote_average          1243317 non-null  float64
 3   vote_count            1243317 non-null  int64  
 4   status                1243317 non-null  object 
 5   release_date          1009119 non-null  object 
 6   revenue               1243317 non-null  int64  
 7   runtime               1243317 non-null  int64  
 8   adult                 1243317 non-null  bool   
 9   backdrop_path         321607 non-null   object 
 10  budget                1243317 non-null  int64  
 11  homepage              130027 non-null   object 
 12  imdb_id               627405 non-null   object 
 13  original_language     1243317 non-null  object 
 14  original_title        1243304 non-

In [34]:
Data = Data.dropna(subset=['title'])
Data['release_date'] = pd.to_datetime(Data['release_date'])
Data = Data.dropna(subset=['release_date'])
Data['release_year'] = Data['release_date'].dt.year


In [36]:
import ast

def extract_genres(x):
    try:
        genres = ast.literal_eval(x)
        return [g['name'] for g in genres]
    except:
        return []

Data['genres'] = Data['genres'].fillna('[]').apply(extract_genres)


In [38]:
Data = Data.drop(['backdrop_path', 'homepage', 'poster_path', 'tagline', 'imdb_id', 'overview', 'keywords'], axis=1)


In [50]:
# Create profit feature
Data['profit'] = Data['revenue'] - Data['budget']

# Create success label (1 = success, 0 = failure)
Data['success'] = Data.apply(lambda x: 1 if x['revenue'] >= 2 * x['budget'] else 0, axis=1)

# Create number of genres
Data['num_genres'] = Data['genres'].apply(lambda x: len(x))

# Filter very large or zero budgets (optional)
Data = Data[Data['budget'] > 10000]
Data = Data[Data['revenue'] > 10000]


In [52]:
features = ['budget', 'runtime', 'popularity', 'vote_average', 
            'vote_count', 'release_year', 'num_genres']
target = 'success'

X = Data[features]
y = Data[target]

# Handle missing values if any
X = X.fillna(0)


In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [56]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [58]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7322239031770046
Confusion Matrix:
 [[796 252]
 [279 656]]
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.76      0.75      1048
           1       0.72      0.70      0.71       935

    accuracy                           0.73      1983
   macro avg       0.73      0.73      0.73      1983
weighted avg       0.73      0.73      0.73      1983



In [62]:
import joblib 
model.fit(X_train, y_train)

# Save model
joblib.dump(model, 'model.pkl')

['model.pkl']